In [4]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
#from visual import *
import numpy as np
import timeit
from math import *

In [10]:
def initialize(speed = 1, N = 100, width = 1000, height = 1000):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    agents = [Point(width*(0.25+0.5*random()), height*(0.25+0.5*random())) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    for i in range(N):
        theta = 2 * np.pi * random()
        speeds[i][0] = speed * np.cos(theta)
        speeds[i][1] = speed * np.sin(theta)

    return agents, speeds, draw_agents(agents, width, height)


def draw_agents(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
            weightedSpeed = a * speeds[nearest_neighbours[i]]
            speeds[i] = speeds[i] + weightedSpeed
            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------


def get_distances(agent, agents):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    
    dists = [(a.getX() - agent.getX())**2 + (a.getY() - agent.getY())**2 for a in agents]
    for i in range(len(dists)):
        if dists[i] == 0:
            dists[i] = 0.1
    return dists


def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def treat_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


#def get_v_avg(speeds, N):
#    return
#   
#def get_potencials(distances, N):
#    potentials = np.zeros(N, N)
#    for i in range(N):
#        for j in range(i+1,N):
#            potentials[i][j] = potentials[j][i] = morse_pot(distances[i][j])
#    return potentials  
#    
#def morse_pot(r, ca = 0.5, cr = 1, la = 3, lr = 0.5):
#    return cr*np.exp(-r/lr) - ca * np.exp(-r)
#
#def weird_paper(agents, N, dt):
#    distances = [get_distances(agent, agents) for agent in agents]
#    potencials = get_potencials(distances, N)
#    for i in range(N):
#            newSpeed = speed[i] + dt * potentials
#            speeds[i] = speeds[i] + weightedSpeed
#            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------
#
#    return

In [22]:
#COUZIN MODEL IMPLEMENTED WITH REPULSE, ATRACT AND ORIENT ZONES SEPARATED (1ST PAPER)
def couzin(agents, speeds, N, s, noise, rr, ro, ra):
    # watch only particles inrepuls
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents)
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue

            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            
            elif not repulsion_flag:

                if distances[j] < ro:
                    o_dir = o_dir + speeds[j]
                
                elif distances[j] < ra:
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
            
        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = o_dir + a_dir
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir

#COUZIN MODEL (2) IMPLEMENTED WITH ATRACT AND ORIENT ZONES TOGETHER (2ND PAPER)
def couzin2(agents, speeds, N, s, noise, rr, roa, orient, atract):
    # watch only particles inrepuls
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents)
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue
            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            elif not repulsion_flag:
                if distances[j] < roa:
                    o_dir = o_dir + speeds[j]
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = orient*o_dir + atract*a_dir
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(tot_dir + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir


def biaser(speeds, N, s, i, prop, bias, dev_bias, rot_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += np.arccos(np.dot(bias,normalized(speeds[i]))) # stupid way of doing an angle
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    print (group_dir)
    return cm
    
def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    for i in range(N):
        distances = get_distances(agents[i], agents)
        tot_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                tot_dir = tot_dir + speeds[j]

        tot_dir = s * normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = tot_dir

def gueron():
    return

In [23]:
def simulate(N_steps, a, dt, N, width, height, s, noise, prop, weight, bias, dev_bias, rot_bias):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    #Watch out!! format of old_cm was giving problems
    #old_cm=Point(0,0)
    #old_cm.setFill("red")
    old_cm=np.array([0.0,0.0])
    
    
    agents, speeds, window = initialize(s, N, width, height)
    for i in range(N_steps):
        next_step(agents, speeds, dt, N)
        import timeit

        start = timeit.default_timer()
        
        ## MODEL CHOSEN (now with couzin 2 with orient and atract zones together)
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, s, noise,200,2500,3000)
        couzin2(agents, speeds, N, s, noise,200,3000, 1.5,1) #(...)last 3 parameters: roa, orient, atract
        #EXTRA ON COUZIN MODEL
        bias = biaser(speeds, N, s, i, prop, bias, dev_bias, rot_bias, weight)
        #INFORMATION TRANSFER: SHAPE & DIRECTION & ALIGNMENT QUALITY 
        #point_cm.undraw()
        old_cm = quality(agents, speeds, N, bias, window, old_cm)
        #print (old_cm)
        
        ## BOUNDARY CONDITIONS
        #treat_boundary(width, height, agents, speeds, N)
        periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    

#simulate(N_steps, a, dt, N, width, height, s, noise, prop, weight, bias, dev_bias, rot_bias)
simulate(3000, 0.1, 2, 50, 900, 700, 2, 0.01, 0.4,0.2, np.array([0.0,-1.0]), 0.1, 1)

[ 445.54607747  356.32271652]
570.506078371
[-0.44252183  0.03585169]
0.443971751362
[-0.31192416 -0.12964944]
0.337795291983
[-0.56242982 -0.33546556]
0.654877432282
[-0.85141016 -1.41203414]
1.648860118
[-0.81210067 -1.06414885]
1.33862626509
[-0.4257963  -0.42717233]
0.603140689698
[-0.1203732  -0.37343824]
0.39235930536
[-0.42016347 -0.49956365]
0.652764264973
[-0.92869464 -0.46472991]
1.03848333189
[-1.41708242 -1.48247386]
2.05081723235
[-0.78553081 -1.26007503]
1.48487296973
[-0.73719997 -0.98908573]
1.23359409026
[-0.94059191 -1.05209976]
1.41125017214
[-1.14728987 -1.59539703]
1.96508669367
[-0.07799514 -1.22409345]
1.2265757294
[-0.45753441 -1.03725963]
1.13368658416
[-0.13159348 -0.96212285]
0.971080439028
[-0.13712191 -1.19062852]
1.19849851959
[-0.45084956 -1.89246143]
1.94542426937
[ 0.13357513 -0.96758231]
0.976758841403
[-0.35177013 -1.29053706]
1.33762032416
[-0.04589172 -1.20674121]
1.20761350909
[ -1.20468965e-04  -8.03654778e-01]
0.803654787283
[-0.57227028 -1.30079

[-0.62838445  0.04311637]
0.629861922293
[-1.40903994  0.55127515]
1.51304257417
[-0.99724072  1.00817911]
1.41806705369
[-1.13049185  0.81948718]
1.39627041013
[-1.19540469  0.30949215]
1.23481891583
[-1.20603009  0.6972097 ]
1.39305776706
[-1.17804219  0.9330505 ]
1.50278629001
[-1.33194255  1.09657253]
1.72526585617
[-1.01018611  1.36852091]
1.70097779362
[-1.18408205  0.98152422]
1.53799873092
[-1.1051426   0.92937685]
1.44398112708
[-0.33477727  0.76457576]
0.834656767167
[-0.55363853  0.57771157]
0.800166405633
[-0.66665959  0.67208665]
0.946644324393
[-0.83978787 -0.13271175]
0.850209427044
[-1.03523975 -0.47251262]
1.13797606192
[-1.10182958  0.41014892]
1.1756915237
[-0.31068962  0.46974595]
0.563195615796
[-0.85267636  0.5635275 ]
1.02206663839
[-0.49108006  0.36637443]
0.612690661737
[-0.36473852  0.69847467]
0.787972749302
[-0.50550925  0.78237694]
0.931479079017
[-0.75785913  0.47923826]
0.89667149409
[-0.39174042  0.32051314]
0.506151390307
[-0.31808288  0.36001762]
0.480

[-0.7140902 -0.4143222]
0.825583248385
[  -0.83843894  643.51612878]
643.516674981
[  -0.78757794 -630.35557113]
630.356063135
[ -2.23352430e-01   6.57365622e+02]
657.365660334
[ -1.65273575e-01  -6.43951251e+02]
643.951272655
[ 0.16929941  0.25291493]
0.304348895347
[-0.7393788  -0.08300368]
0.744023270129
[-0.40564902  0.02928403]
0.406704663641
[-0.84547063 -0.21512945]
0.872411181909
[-0.92566404 -0.54373857]
1.07354810635
[-0.58855638 -0.36078818]
0.690338118354
[-0.60469432 -0.50686452]
0.789029066209
[ 0.3650913  -0.13427745]
0.389001400275
[-0.50542996 -0.80253487]
0.94843115664
[ 0.10653652 -1.2403011 ]
1.24486820624
[ 0.14889252 -1.83407944]
1.84011314594
[ 0.35686849 -1.289593  ]
1.3380602477
[ 0.42775139 -0.91006295]
1.00557735988
[  5.24738912e-01   6.84437827e+02]
684.438027876
[   0.92770947 -672.80116905]
672.801808645
[ 0.59562135 -0.99907521]
1.16314920374
[  6.63267501e-02   4.47552478e+02]
447.552482691
[  2.25581036e-01  -4.35030365e+02]
435.030423112
[  1.78308462

[  7.86473661e-01  -3.79024126e-04]
0.786473751842
[ 1.11201182 -0.32128758]
1.15749557083
[ 0.34494195 -0.3762976 ]
0.51047510365
[ 0.20327599 -0.54478981]
0.581478344813
[ 0.17403008 -0.35554622]
0.395852984724
[-0.93856547 -0.32791688]
0.994200492532
[-0.74494008 -0.58427088]
0.946735540371
[-1.06394427  0.38877801]
1.13275140276
[-1.59010679 -0.47830841]
1.66048743812
[-0.52043096 -0.46557771]
0.698291476219
[-0.96547073 -1.28492536]
1.60722334915
[-1.13334343 -0.8237946 ]
1.40110844546
[-1.53302251 -1.00762585]
1.8345211518
[-0.93008041 -0.72513881]
1.17935399853
[-0.58061338 -0.5627795 ]
0.808599195375
[-0.32854823 -0.43648387]
0.546316859517
[-0.77639729 -0.64894406]
1.01188989007
[-0.23495155 -0.17321981]
0.291902950206
[-0.1016841  -0.62313987]
0.631381778469
[-0.88402035 -0.2318765 ]
0.913924884593
[-1.00405742 -0.60587814]
1.17269758542
[-1.08192667 -0.73664603]
1.30889750795
[-1.03847575 -0.74085032]
1.27565319798
[-0.14616827 -0.9382251 ]
0.949542787047
[-0.33950573 -0.825

[ 0.16821673  0.62166327]
0.644020259858
[ 0.55585793  0.06666141]
0.559840850699
[ 0.14219151 -0.31640916]
0.346890730765
[-0.29311145 -0.15503677]
0.331588187207
[-0.37568739 -0.13372964]
0.398778927803
[-0.74856452 -0.73925073]
1.05206486947
[-1.0070261 -0.6883313]
1.21979570157
[-1.11581084 -0.94813872]
1.46424070985
[-0.68932174 -1.38899929]
1.55063970583
[-0.25452535 -1.33977101]
1.36373366796
[-0.00838899 -1.05048527]
1.05051876645
[-0.05929305 -0.8531874 ]
0.855245232766
[-0.36492607 -0.90862984]
0.979172718717
[-0.39501328 -1.36788646]
1.42377978038
[-1.05523601 -1.66965653]
1.97516479384
[-1.18428799 -0.59269121]
1.32431903263
[-0.30320119 -0.73603238]
0.796036824803
[ 0.282677   -1.56668563]
1.5919830891
[ 0.82423178 -0.87004652]
1.19847360497
[ 0.75231115 -0.41378758]
0.85859898883
[ 1.55287449 -0.7697669 ]
1.73319366355
[ 0.74589709 -0.38367786]
0.83879149118
[ 0.2894605   0.09952384]
0.306092107119
[-0.16875249  0.30354435]
0.347298970063
[ 0.00670239  0.06168497]
0.06204

...

[  1.45262277e-01   3.34069550e+02]
334.069581262
[   0.91780144 -254.02062975]
254.0222878
[   0.72522317 -379.48880531]
379.48949828
[   0.46989239  320.59069465]
320.591039011
[ 0.61844763 -0.7859193 ]
1.00007330632
[ 0.19213543 -1.46234118]
1.47490940213
[ 0.010891   -1.46951043]
1.46955078907
[-179.73914603   -1.17085111]
179.742959547
[ 161.91733339   -1.13850281]
161.921335962
[ 0.12357548 -1.49096629]
1.49607866433
[-0.38713334 -1.45816029]
1.5086761237
[-0.85192937 -2.07136946]
2.23972210436
[  -0.22930332 -142.29439901]
142.294583771
[  -0.35838209  124.86837412]
124.868888411
[ 0.06731031 -1.15707294]
1.15902910179
[-0.42359033 -1.45391708]
1.51436576058
[-0.09493067 -1.19755768]
1.20131437194
[-0.27650024 -1.00096294]
1.03845037933
[  -0.73725077  139.2600091 ]
139.261960607
[  -1.00014129 -126.67491743]
126.678865591
[  -0.82707298 -141.43768239]
141.440100572
[ -4.00971957e-01   6.70016482e+02]
670.01660243
[  1.39495905e-01  -6.59383540e+02]
659.38355517
[  -0.66510022  

[-0.31222082 -0.24190526]
0.394968347599
[-0.89167635 -1.09913729]
1.41534076836
[-0.8253142  -1.15134993]
1.41659810802
[-0.51366827 -1.40447173]
1.49545842979
[ 0.07282812 -0.55334397]
0.55811601583
[ 0.01344558  0.15369673]
0.154283722742
[ 0.69808772  0.12521411]
0.709228484498
[ 0.08348297 -0.29785044]
0.309328771427
[ 0.04667659 -0.83500698]
0.836310560278
[ 0.07077429 -0.91887393]
0.921595519814
[-0.19529771 -1.63812554]
1.64972618725
[-0.08494648 -1.78760479]
1.78962196907
[ 0.22386855 -1.38935966]
1.40728014389
[ 0.05152273 -2.07475939]
2.07539902379
[-0.11835164 -2.01928723]
2.02275258622
[-0.07472552 -1.61070655]
1.61243899164
[-0.34211532 -1.42781176]
1.46822659116
[-0.99741176 -1.08834445]
1.47625331939
[-0.2896414  -1.82941856]
1.85220527735
[ 0.26305494 -1.33446512]
1.36014523461
[ 0.3972382  -1.27047503]
1.33112921588
[-0.30856008 -1.36903789]
1.40337951891
[-0.64853686 -1.43811904]
1.57758880181
[-0.45723584 -1.30871554]
1.38629037095
[ 0.07972357 -0.61179008]
0.616962

[-0.0751936   0.48006775]
0.485920898921
[ 0.03254212 -0.21082102]
0.213317822569
[ 0.31210019 -0.51829745]
0.605011386905
[ 0.15651504 -0.70551605]
0.722668566904
[-0.24740429 -0.93161333]
0.963904700931
[-0.39890568 -1.08644764]
1.15736520873
[-0.4504459  -1.22719644]
1.30725384123
[ -5.63728990e-01   5.87176428e+02]
587.176698385
[ -1.37165489e-01  -5.75340840e+02]
575.34085682
[  2.47426697e-02   6.43216522e+02]
643.216522942
[ -4.09255536e-01  -6.30295446e+02]
630.295578909
[ -9.77004903e-02   6.85848842e+02]
685.848849343
[ -2.00208542e-02  -6.72352942e+02]
672.352942526
[-0.82361974 -0.29520233]
0.874925081803
[  -1.24212097  503.26409598]
503.265628834
[  -0.88146172 -490.56481447]
490.565606385
[  -1.10613176  573.39058893]
573.391655855
[  -0.78861236 -560.657005  ]
560.657559627
[-0.66950028 -0.34335124]
0.752409928731
[-0.35914857 -0.5879425 ]
0.688958693033
[-0.48221758 -0.41467155]
0.635992367251
[-0.3165162  -0.64012898]
0.714106159753
[  1.28933560e-01   6.15548802e+02]

[ 1.05452573 -0.09752607]
1.05902589851
[ 1.24072032 -0.35341564]
1.29007345906
[ 0.75898875 -0.69553639]
1.02948277873
[ 0.39228872 -1.35773208]
1.41326813757
[ 0.44367851 -1.88366398]
1.93521073806
[ 0.51265101 -1.84118596]
1.91122390398
[  -0.39844124  361.89759823]
361.897817568
[  -0.84341037 -351.70835727]
351.709368534
[-0.2543521  -0.64826871]
0.696381587576
[-0.51046669 -0.97033961]
1.09641925948
[  -0.4839138   419.21458494]
419.214864241
[  -0.76117077 -406.71811658]
406.718828838
[-0.53616326 -0.48279833]
0.721502086548
[  -0.91504405  111.97668432]
111.980423008
[ -0.97620907 -98.75199943]
98.7568244501
[ -0.74328269  12.47658302]
12.4987036483
[-0.10679369 -1.71144433]
1.71477303913
[ -0.8864213  -57.97538915]
57.98216527
[ -0.60843161  67.76986055]
67.7725917129
[-0.46963908 -1.22014965]
1.30741196486
[-0.32227676 -1.02760014]
1.0769514218
[-0.26993553 -1.3599563 ]
1.38648704987
[-0.43510768 -1.47639816]
1.53917842486
[-0.35918254 -1.2886418 ]
1.33776297733
[-0.22970553 

[-449.42223513   -0.68542746]
449.422757818
[-719.49729219   -1.11365919]
719.498154071
[ 702.07283554   -0.84956695]
702.073349558
[ -6.29798137e+02  -4.79467472e-01]
629.798319484
[ 612.15089402   -1.2819661 ]
612.152236362
[ 0.4414269  -1.40421929]
1.47196790952
[-0.19669786 -1.34823724]
1.36251007058
[ 0.16823773 -0.80865716]
0.82597235585
[ 0.65280977 -0.70501801]
0.960838692555
[ 0.88550724 -1.02492807]
1.35447429652
[-0.45627338 -0.84648458]
0.961624428584
[-0.84508334 -0.66123717]
1.07303329247
[-1.32396523 -1.24563043]
1.81782262355
[-1.22383202 -1.1564091 ]
1.68375973059
[-0.52261088 -0.66058313]
0.842313606778
[-0.29006103 -0.67608321]
0.73567921888
[-0.96985972 -1.12531917]
1.48558780198
[-0.59595721 -0.71355134]
0.929688393824
[-0.95819184 -0.73097039]
1.20517605325
[-0.78369206 -0.7990481 ]
1.11921897378
[-0.30633233 -1.3473252 ]
1.3817107834
[-0.21061174 -1.1704985 ]
1.18929560839
[ 0.01309406 -1.50437638]
1.50443336868
[-0.00482919 -2.00519985]
2.00520566126
[-0.2483353